In [1]:
import pandas as pd
import math
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats.stats import pearsonr


In [2]:
df = pd.read_csv('dataset/matches_datacleaning.csv', index_col = 0)
pd.set_option('display.max_columns', None) # to visualize all the columns
df.info()


C:\Users\Gerlando\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
Int64Index: 184890 entries, 0 to 186072
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   tourney_id          184890 non-null  object 
 1   tourney_name        184890 non-null  object 
 2   surface             184728 non-null  object 
 3   draw_size           184890 non-null  float64
 4   tourney_level       184890 non-null  object 
 5   tourney_date        184890 non-null  object 
 6   match_num           184890 non-null  float64
 7   winner_id           184890 non-null  float64
 8   winner_entry        25701 non-null   object 
 9   winner_name         184890 non-null  object 
 10  winner_hand         184890 non-null  object 
 11  winner_ht           48560 non-null   float64
 12  winner_ioc          184890 non-null  object 
 13  winner_age          182080 non-null  float64
 14  loser_id            184890 non-null  float64
 15  loser_entry         43927 non-null

In [3]:
df['tourney_date'] = pd.to_datetime(df['tourney_date'])
df['tourney_year'] = df['tourney_date'].dt.year

### functions

In [4]:
def create_new_colums(winner_attr, loser_attr, attr):
    col1=df[['winner_name',winner_attr]]
    col2=df[['loser_name',loser_attr]]
    col2.rename(columns={'loser_name': 'winner_name'}, inplace = True)
    col2.rename(columns={loser_attr: winner_attr}, inplace = True)
    name2 = col1.append(col2) 
    name2.rename(columns={'winner_name': 'name',winner_attr:attr}, inplace = True)
    name2.drop_duplicates(inplace=True)
    return name2


In [5]:
def create_df(series):
    tmp = pd.DataFrame(series)
    tmp.reset_index(inplace=True)
    return tmp

def count_occ(df, attr_to_count):
    tmp = df[attr_to_count].value_counts(dropna=False)
    tmp = create_df(tmp)
    return tmp

## Dataframe with Players profile

#### matches' number per player

In [6]:
#matches won per player
num_matches_won = count_occ(df, 'winner_name')
num_matches_won.set_axis(['name', 'num_matches'], axis=1, inplace=True)

#matches loss per player
num_matches_loss = count_occ(df, 'loser_name')
num_matches_loss.set_axis(['name', 'num_matches'], axis=1, inplace=True)

#total matches per player
num_matches = num_matches_won.append(num_matches_loss)
num_matches = num_matches.groupby(['name']).num_matches.sum()
num_matches = create_df(num_matches)
num_matches.set_axis(['name', 'num_matches'], axis=1, inplace=True)

#drop players with few matches
num_matches = num_matches[num_matches['num_matches'] > 2]

#add ratio
num_matches_won.set_axis(['name', 'num_matches_won'], axis=1, inplace=True) #rename attribute 'num_matches' to 'num_matches_won', otherwise we could have a conflict
num_matches = pd.merge(num_matches, num_matches_won, how = 'left', on = 'name')
num_matches['ratio'] = num_matches['num_matches_won']/num_matches['num_matches']
num_matches.fillna(0, inplace = True)

del num_matches['num_matches_won'] #reduntant column

#RENAME our database
player_profile = num_matches
player_profile

,name,num_matches,ratio
0,AALIYA EBRAHIM,9,0.222222
1,AALIYAH HOHMANN,3,0.333333
2,AALYKA EBRAHIM,4,0.000000
3,AANU AYEGBUSI,6,0.500000
4,AARON ADDISON,11,0.181818
...,...,...,...
6640,ZUZANA BOHATOVA,6,0.000000
6641,ZUZANA PAVELEKOVA,4,0.500000
6642,ZUZANA ZALABSKA,3,0.666667
6643,ZUZANA ZLOCHOVA,179,0.519553


## Statistics of players during period

In [7]:
years=df['tourney_year'].unique()
years.sort()
for year in years:
    num_matches_year = df[df['tourney_year']==year] #takes only matches of a year
    num_m_y = 'num_matches_'+str(year)
    
    #matches won per player
    num_matches_won = count_occ(num_matches_year, 'winner_name')
    num_matches_won.set_axis(['name', num_m_y], axis=1, inplace=True)
    
    #matches loss per player
    num_matches_loss = count_occ(num_matches_year, 'loser_name')
    num_matches_loss.set_axis(['name', num_m_y], axis=1, inplace=True)
    
    #total matches per player
    num_matches = num_matches_won.append(num_matches_loss)
    num_matches = num_matches.groupby(['name'])[num_m_y].sum()
    num_matches = create_df(num_matches)
    num_matches.set_axis(['name', num_m_y], axis=1, inplace=True)
    
    #add ratio
    num_matches_won.set_axis(['name', 'num_matches_won'], axis=1, inplace=True) #rename attribute 'num_matches' to 'num_matches_won', otherwise we could have a conflict
    num_matches = pd.merge(num_matches, num_matches_won, how = 'left', on = 'name')
    num_matches['ratio_'+str(year)] = num_matches['num_matches_won']/num_matches[num_m_y]
    num_matches.fillna(0, inplace = True)
    del num_matches['num_matches_won']
    
    #add to our player profile ratio and matches of current year
    player_profile = pd.merge(player_profile, num_matches, how = 'left', on = 'name')
    player_profile.fillna(0, inplace = True) #players with no matches during a year

player_profile
    

,name,num_matches,ratio,num_matches_2016,ratio_2016,num_matches_2017,ratio_2017,num_matches_2018,ratio_2018,num_matches_2019,ratio_2019,num_matches_2020,ratio_2020,num_matches_2021,ratio_2021
0,AALIYA EBRAHIM,9,0.222222,0.0,0.000000,0.0,0.0000,0.0,0.000000,8.0,0.250000,1.0,0.0,0.0,0.000000
1,AALIYAH HOHMANN,3,0.333333,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,3.0,0.333333
2,AALYKA EBRAHIM,4,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.000000,4.0,0.000000,0.0,0.0,0.0,0.000000
3,AANU AYEGBUSI,6,0.500000,0.0,0.000000,0.0,0.0000,1.0,0.000000,5.0,0.600000,0.0,0.0,0.0,0.000000
4,AARON ADDISON,11,0.181818,6.0,0.333333,0.0,0.0000,1.0,0.000000,3.0,0.000000,1.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6640,ZUZANA BOHATOVA,6,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.000000,5.0,0.000000,0.0,0.0,1.0,0.000000
6641,ZUZANA PAVELEKOVA,4,0.500000,0.0,0.000000,0.0,0.0000,0.0,0.000000,3.0,0.666667,1.0,0.0,0.0,0.000000
6642,ZUZANA ZALABSKA,3,0.666667,3.0,0.666667,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
6643,ZUZANA ZLOCHOVA,179,0.519553,51.0,0.627451,16.0,0.1875,44.0,0.636364,50.0,0.500000,10.0,0.4,8.0,0.125000


## Player's Hand

In [8]:
df_hand=create_new_colums('winner_hand','loser_hand','hand')
df_hand.at[180480, 'hand'] = 'R'#AGGIUSTARE QUESTA COSA
df_hand.drop_duplicates(inplace=True)

player_profile=pd.merge(player_profile, df_hand, how='left', on='name')
player_profile

c:\users\gerlando\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,name,num_matches,ratio,num_matches_2016,ratio_2016,num_matches_2017,ratio_2017,num_matches_2018,ratio_2018,num_matches_2019,ratio_2019,num_matches_2020,ratio_2020,num_matches_2021,ratio_2021,hand
0,AALIYA EBRAHIM,9,0.222222,0.0,0.000000,0.0,0.0000,0.0,0.000000,8.0,0.250000,1.0,0.0,0.0,0.000000,R
1,AALIYAH HOHMANN,3,0.333333,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,3.0,0.333333,U
2,AALYKA EBRAHIM,4,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.000000,4.0,0.000000,0.0,0.0,0.0,0.000000,U
3,AANU AYEGBUSI,6,0.500000,0.0,0.000000,0.0,0.0000,1.0,0.000000,5.0,0.600000,0.0,0.0,0.0,0.000000,U
4,AARON ADDISON,11,0.181818,6.0,0.333333,0.0,0.0000,1.0,0.000000,3.0,0.000000,1.0,0.0,0.0,0.000000,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6640,ZUZANA BOHATOVA,6,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.000000,5.0,0.000000,0.0,0.0,1.0,0.000000,U
6641,ZUZANA PAVELEKOVA,4,0.500000,0.0,0.000000,0.0,0.0000,0.0,0.000000,3.0,0.666667,1.0,0.0,0.0,0.000000,U
6642,ZUZANA ZALABSKA,3,0.666667,3.0,0.666667,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,R
6643,ZUZANA ZLOCHOVA,179,0.519553,51.0,0.627451,16.0,0.1875,44.0,0.636364,50.0,0.500000,10.0,0.4,8.0,0.125000,R


## Player's Gender

In [9]:
df_gender = create_new_colums('winner_gender','loser_gender','gender')
player_profile = pd.merge(player_profile, df_gender, how='left', on='name')
player_profile

c:\users\gerlando\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,name,num_matches,ratio,num_matches_2016,ratio_2016,num_matches_2017,ratio_2017,num_matches_2018,ratio_2018,num_matches_2019,ratio_2019,num_matches_2020,ratio_2020,num_matches_2021,ratio_2021,hand,gender
0,AALIYA EBRAHIM,9,0.222222,0.0,0.000000,0.0,0.0000,0.0,0.000000,8.0,0.250000,1.0,0.0,0.0,0.000000,R,F
1,AALIYAH HOHMANN,3,0.333333,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,3.0,0.333333,U,F
2,AALYKA EBRAHIM,4,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.000000,4.0,0.000000,0.0,0.0,0.0,0.000000,U,F
3,AANU AYEGBUSI,6,0.500000,0.0,0.000000,0.0,0.0000,1.0,0.000000,5.0,0.600000,0.0,0.0,0.0,0.000000,U,F
4,AARON ADDISON,11,0.181818,6.0,0.333333,0.0,0.0000,1.0,0.000000,3.0,0.000000,1.0,0.0,0.0,0.000000,R,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6640,ZUZANA BOHATOVA,6,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.000000,5.0,0.000000,0.0,0.0,1.0,0.000000,U,F
6641,ZUZANA PAVELEKOVA,4,0.500000,0.0,0.000000,0.0,0.0000,0.0,0.000000,3.0,0.666667,1.0,0.0,0.0,0.000000,U,F
6642,ZUZANA ZALABSKA,3,0.666667,3.0,0.666667,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,R,F
6643,ZUZANA ZLOCHOVA,179,0.519553,51.0,0.627451,16.0,0.1875,44.0,0.636364,50.0,0.500000,10.0,0.4,8.0,0.125000,R,F


## Player's nationality

In [10]:
df_ioc = create_new_colums('winner_ioc','loser_ioc','ioc')
df_ioc.drop_duplicates(inplace=True)

player_profile = pd.merge(player_profile, df_ioc, how='left', on='name')
player_profile

c:\users\gerlando\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,name,num_matches,ratio,num_matches_2016,ratio_2016,num_matches_2017,ratio_2017,num_matches_2018,ratio_2018,num_matches_2019,ratio_2019,num_matches_2020,ratio_2020,num_matches_2021,ratio_2021,hand,gender,ioc
0,AALIYA EBRAHIM,9,0.222222,0.0,0.000000,0.0,0.0000,0.0,0.000000,8.0,0.250000,1.0,0.0,0.0,0.000000,R,F,IND
1,AALIYAH HOHMANN,3,0.333333,0.0,0.000000,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,3.0,0.333333,U,F,DEU
2,AALYKA EBRAHIM,4,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.000000,4.0,0.000000,0.0,0.0,0.0,0.000000,U,F,IND
3,AANU AYEGBUSI,6,0.500000,0.0,0.000000,0.0,0.0000,1.0,0.000000,5.0,0.600000,0.0,0.0,0.0,0.000000,U,F,NGR
4,AARON ADDISON,11,0.181818,6.0,0.333333,0.0,0.0000,1.0,0.000000,3.0,0.000000,1.0,0.0,0.0,0.000000,R,M,AUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6640,ZUZANA BOHATOVA,6,0.000000,0.0,0.000000,0.0,0.0000,0.0,0.000000,5.0,0.000000,0.0,0.0,1.0,0.000000,U,F,CZE
6641,ZUZANA PAVELEKOVA,4,0.500000,0.0,0.000000,0.0,0.0000,0.0,0.000000,3.0,0.666667,1.0,0.0,0.0,0.000000,U,F,SVK
6642,ZUZANA ZALABSKA,3,0.666667,3.0,0.666667,0.0,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,R,F,CZE
6643,ZUZANA ZLOCHOVA,179,0.519553,51.0,0.627451,16.0,0.1875,44.0,0.636364,50.0,0.500000,10.0,0.4,8.0,0.125000,R,F,SVK


## ages of players CONTINUA A SISTEMARE DA QUA:

In [11]:
LoserId_name = df.groupby(['winner_name']).winner_age.nunique() 
LoserId_name = LoserId_name[LoserId_name > 1] 

df[df.winner_name.isin(LoserId_name.index)].groupby(['winner_name']).winner_age.unique()

winner_name
ABBIE MYERS                        [23.0, 22.0, 24.0, 25.0, 26.0]
ABHINAV SANJEEV SHANMUGAM                      [18.0, 19.0, 21.0]
ABIGAIL DESIATNIKOV                                  [17.0, 15.0]
ABIGAIL RENCHELI                                     [17.0, 18.0]
ABIGAIL TERE APISAH                [25.0, 24.0, 26.0, 27.0, 28.0]
                                             ...                 
ZOE KRUGER                   [15.0, 14.0, 16.0, 17.0, 19.0, 18.0]
ZOE RICHARD                              [19.0, 20.0, 21.0, 22.0]
ZOZIYA KARDAVA                     [16.0, 17.0, 18.0, 20.0, 19.0]
ZSOMBOR PIROS                            [18.0, 19.0, 22.0, 20.0]
ZUZANA ZLOCHOVA              [27.0, 28.0, 26.0, 30.0, 29.0, 31.0]
Name: winner_age, Length: 3643, dtype: object

In [12]:
gg=df

gg['winner_birth']=(gg['tourney_date'].str[:4].astype('int32')-gg['winner_age'])
gg['loser_birth']=(gg['tourney_date'].str[:4].astype('int32')-gg['loser_age'])
#print(var)
print()

AttributeError: Can only use .str accessor with string values!

In [13]:
winner_birth=gg.groupby(['winner_name']).winner_birth.min() #print(gg.groupby(['winner_name']).birth.min())
winner_birth = pd.DataFrame(winner_birth)
winner_birth.reset_index(inplace=True)
winner_birth.rename(columns={'winner_name': 'name', 'winner_birth': 'birth'}, inplace = True)

loser_birth=gg.groupby(['loser_name']).loser_birth.min() #print(gg.groupby(['loser_name']).birth.min())
loser_birth = pd.DataFrame(loser_birth)
loser_birth.reset_index(inplace=True)
loser_birth.rename(columns={'loser_name': 'name', 'loser_birth': 'birth'}, inplace = True)

name_birth=winner_birth.append(loser_birth)
name_birth.dropna(inplace=True)
name_birth.drop_duplicates(inplace=True)
name_birth=name_birth.groupby(name_birth['name']).birth.min()

name_birth = pd.DataFrame(name_birth)
name_birth.reset_index(inplace=True)
#print(name_birth)


In [14]:
new_df=pd.merge(new_df, name_birth, how='left', on='name')
print(new_df)

#controllare perchè l'anno non è intero

                     name  num_match hand gender  ioc   birth
0           KEI NISHIKORI        280    R      M  JPN  1980.5
1         DANIIL MEDVEDEV        376    R      M  RUS  1981.5
2      JO-WILFRIED TSONGA        194    R      M  FRA  1984.0
3           JEREMY CHARDY        278    R      M  FRA  1986.0
4          ALEX DE MINAUR        267    R      M  AUS  1998.0
...                   ...        ...  ...    ...  ...     ...
10093    DILARA SULTANOVA          1    U      F  TKM     NaN
10094    MADELEINE JESSUP          1    U      F  USA     NaN
10095          KATIE CODD          1    U      F  USA     NaN
10096           AMY HUANG          1    U      F  USA     NaN
10097     CHLOE HENDERSON          1    U      F  USA     NaN

[10098 rows x 6 columns]


## height of players

In [15]:
value=create_new_colums('winner_ht','loser_ht','ht')
value.dropna(inplace=True)
value.drop_duplicates(inplace=True)

name_ht = value.groupby(['name']).ht.max() 

new_df=pd.merge(new_df, name_ht, how='left', on='name')
print(new_df)

#completare aggiungendo l'altezza media tramite le nazionalità dei giocatori

                     name  num_match hand gender  ioc   birth     ht
0           KEI NISHIKORI        280    R      M  JPN  1980.5  178.0
1         DANIIL MEDVEDEV        376    R      M  RUS  1981.5  198.0
2      JO-WILFRIED TSONGA        194    R      M  FRA  1984.0  188.0
3           JEREMY CHARDY        278    R      M  FRA  1986.0  188.0
4          ALEX DE MINAUR        267    R      M  AUS  1998.0  183.0
...                   ...        ...  ...    ...  ...     ...    ...
10093    DILARA SULTANOVA          1    U      F  TKM     NaN    NaN
10094    MADELEINE JESSUP          1    U      F  USA     NaN    NaN
10095          KATIE CODD          1    U      F  USA     NaN    NaN
10096           AMY HUANG          1    U      F  USA     NaN    NaN
10097     CHLOE HENDERSON          1    U      F  USA     NaN    NaN

[10098 rows x 7 columns]


/Users/domenico/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
